In [67]:
battleField =   [[1, 0, 0, 0, 0, 1, 1, 0, 0, 0],
                 [1, 0, 1, 0, 0, 0, 0, 0, 1, 0],
                 [1, 0, 1, 0, 1, 1, 1, 0, 1, 0],
                 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [68]:
class Ship:
    def __init__(self, type, size, count):
        self.type = type
        self.size = size
        self.count = count
    def __repr__(self):
        return f"{self.type}: size = {self.size}, cur count = {self.count}"

class Fleet:    
    def get_ships(self):
        return [self.btl, self.csr, self.dstr, self.sub]
    
    def check_counts(self):
        if self.btl.count != 1:
            return False
        elif self.csr.count != 2:
            return False
        elif self.dstr.count != 3:
            return False
        elif self.sub.count != 4:
            return False
        else:
            return True

    def __init__(self):
        self.btl = Ship("battleship", 4, 0)
        self.csr = Ship("cruiser", 3, 0)
        self.dstr = Ship("destroyer", 2, 0)
        self.sub = Ship("submarine", 1, 0)
        self.ships = self.get_ships()

    def add_ship(self, size):
        if size == 2:
            self.dstr.count += 1
        elif size == 3:
            self.csr.count += 1
        elif size == 4:
            self.btl.count += 1

In [69]:
def set_idx(r = range(10), grid = False):
    if grid:
        return [[(i, j) for j in r] for i in r]
    else:
        return [(i, j) for j in r for i in r]
idx = set_idx()

class Point:
    def get_adj(self, subset = "all"):
        if subset == "all":
            return [val for val in idx if (abs(self.row - val[0]) <= 1 and abs(self.col - val[-1]) <= 1)]
        elif subset == "diag":
            return [val for val in idx if (abs(self.row - val[0]) == 1 and abs(self.col - val[-1]) == 1)]
        elif subset == "adj":
            return [val for val in idx if (abs(self.row - val[0]) + abs(self.col - val[-1]) == 1)]

    def __init__(self, row, col, isship):
        self.row = row
        self.col = col
        self.coords = (row, col)
        self.isship = isship
        self.diag = self.get_adj(subset="diag")
        self.adj = self.get_adj(subset="adj")
    
    def __repr__(self):
        return str(self.isship)

class Field:
    
    def set_points(self):
        points = set_idx()
        for i, coord in enumerate(self.coords):
            row = coord[0]
            col = coord[-1]
            isship = self.field[row][col]
            points[i] = Point(row, col, isship)
        return points
    
    def get_point(self, coord):
        for point in self.points:
            if point.coords == coord:
                return point
        raise ValueError("point not found with set coordinates")

    
    def __init__(self, field):
        self.field = field
        self.coords = set_idx()
        self.points = self.set_points()
        self.ship_points = [x for x in self.points if x.isship == 1]
        self.ship_coords = [x.coords for x in self.ship_points]

    def __repr__(self):
        out_str = ""
        for i, row in enumerate(self.field):
            if i > 0:
                out_str = out_str + "\n"
            row_str = "  ".join([str(x) for x in row])
            out_str = out_str + row_str
        return out_str

    def check_corners(self):
        for point in self.ship_points:
            diags = point.diag
            is_diag_ship = sum([1 if diag in self.ship_coords else 0 for diag in diags])
            if is_diag_ship > 0:
                return False
        return True



def check_ships(f: Field):
    fleet = Fleet()
    ship_points = f.ship_points
    ship_adj_coords = link([x.adj for x in ship_points])
    cur_ship_coords = list(f.ship_coords)
    sub_coords = [x for x in cur_ship_coords if x not in ship_adj_coords]
    fleet.sub.count += len(sub_coords)
    
    cur_ship_coords = [x for x in ship_adj_coords if (x in cur_ship_coords and x not in sub_coords)]

    non_sub_points = [s for s in ship_points if s.coords not in sub_coords]
    
    adj_coord_dict = {}
    for point in non_sub_points:
        adj_coord_dict[point.coords] = [x for x in point.adj if x in cur_ship_coords]

    all_ships = []
    for l1 in non_sub_points:
        for l2 in [x for x in non_sub_points if (x.coords in l1.adj)]:           
            for l3 in [y for y in non_sub_points if (y.coords in l2.adj)]:
                for l4 in [z for z in non_sub_points if (z.coords in l3.adj)]:
                    ship_coords = [str(x) for x in [l1.coords, l2.coords, l3.coords, l4.coords]]
                    ship_coords.sort()
                    ship = list(set(ship_coords))
                    if ship not in all_ships:
                        all_ships.append(ship)
    fours = [ship for ship in all_ships if len(ship)==4]
    fleet.btl.count = len(fours)
    used_coords = link(fours)

    threes = [ship for ship in all_ships if (len(ship)==3 and sum([1 if x in used_coords else 0 for x in ship])==0)]
    fleet.csr.count = len(threes)
    used_coords = used_coords + link(threes)
    
    twos = [ship for ship in all_ships if (len(ship)==2 and sum([1 if x in used_coords else 0 for x in ship])==0)]
    fleet.dstr.count = len(twos)    
    
    return fleet.check_counts()

def validate_battlefield(field):
    f = Field(field)
    if not f.check_corners():
        return False
    return check_ships(f)


validate_battlefield(battleField)


[(1, 0), (0, 0), (2, 0), (1, 0), (3, 0), (2, 0), (2, 2), (1, 2), (7, 4), (2, 5), (5, 5), (7, 3), (0, 6), (2, 4), (2, 6), (5, 4), (5, 6), (0, 5), (2, 5), (5, 5), (2, 8), (1, 8)]


False